In [1]:
import os
import itertools
import pandas as pd
import numpy as np
import torch
from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler, Diff
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import warnings
import traceback

warnings.filterwarnings("ignore")

# Suppress PyTorch Lightning logs
import pytorch_lightning as pl
pl.utilities.rank_zero_info = lambda *args, **kwargs: None
pl.utilities.rank_zero_warn = lambda *args, **kwargs: None
pl.utilities.rank_zero_debug = lambda *args, **kwargs: None

# ===================================================================
# CONFIGURATION
# ===================================================================

START_FROM_RUN = 1 

DATASETS = {
    "IHSG": r"C:\wilson\Thesis\Data\final_data\ihsg_final.csv",
    "LQ45": r"C:\wilson\Thesis\Data\final_data\lq45_final.csv",
    "Kompas100": r"C:\wilson\Thesis\Data\final_data\kompas100_final.csv"
}

SCENARIOS = {
    "Short-Term Sprint": {"window": 30, "horizon": 1},
    "Medium-Term Pace": {"window": 60, "horizon": 5},
    "Long-Term Marathon": {"window": 90, "horizon": 10},
}

COVARIATE_SETS = {
    "None": [],
    "Domestic": ["close_inflation", "close_gdp", "close_cpi", "close_unemployment"],
    "Global": ["close_usd", "close_brent", "close_wti"],
    "All": ["close_inflation", "close_gdp", "close_cpi", "close_unemployment",
            "close_usd", "close_brent", "close_wti"],
    "Inflation": ["close_inflation"],
    "GDP": ["close_gdp"],
    "CPI": ["close_cpi"],
    "Unemployment": ["close_unemployment"],
    "USD": ["close_usd"],
    "Brent": ["close_brent"],
    "WTI": ["close_wti"]
}

TFT_CONFIG = {
    "dropout": 0.3,
    "hidden_size": 128,
    "num_attention_heads": 4,
    "batch_size": 64,
    "learning_rate": 0.001,
    "n_epochs": 100
}

# Paths
BASE_RESULT_DIR = r"C:\wilson\Thesis\Results"
RESULTS_SUMMARY_PATH = os.path.join(BASE_RESULT_DIR, "tft_results_summary_logret.csv")
PREDICTION_DIR = os.path.join(BASE_RESULT_DIR, "tft_results_logret")

os.makedirs(BASE_RESULT_DIR, exist_ok=True)
os.makedirs(PREDICTION_DIR, exist_ok=True)

# ===================================================================
# HELPER FUNCTION
# ===================================================================

def train_and_evaluate_tft(df, target_col, covariate_cols, input_chunk_length, output_chunk_length,
                           test_ratio=0.2):
    try:
        # 1. Sort and prepare target
        df = df.sort_values("date").reset_index(drop=True)
        target_series = TimeSeries.from_dataframe(df, time_col="date", value_cols=target_col,
                                                  fill_missing_dates=True, freq="B")
        # Fill missing values
        target_series = TimeSeries.from_dataframe(
            target_series.to_dataframe().fillna(method="ffill").fillna(method="bfill")
        )

        # 2. Prepare covariates
        covariate_series = None
        if covariate_cols:
            covariate_series = TimeSeries.from_dataframe(df, time_col="date", value_cols=covariate_cols,
                                                         fill_missing_dates=True, freq="B")
            covariate_series = TimeSeries.from_dataframe(
                covariate_series.to_dataframe().fillna(method="ffill").fillna(method="bfill")
            )

        # 3. TRANSFORM: Log -> Diff (Log Returns)
        # We perform this on the WHOLE series before splitting to ensure consistency
        ts_log = target_series.map(np.log)
        differencer = Diff(lags=1)
        ts_log_diff = differencer.fit_transform(ts_log)

        # 4. Train/test split (On the transformed data)
        # Note: differencing removes the first point, so we align carefully
        train_cutoff = int(len(ts_log_diff) * (1 - test_ratio))
        target_train, target_test = ts_log_diff[:train_cutoff], ts_log_diff[train_cutoff:]
        
        # Split covariates (aligning dates)
        cov_train, cov_test = None, None
        if covariate_series:
            # Slice covariates to match the transformed target's time index
            cov_aligned = covariate_series.slice_intersect(ts_log_diff)
            cov_train, cov_test = cov_aligned[:train_cutoff], cov_aligned[train_cutoff:]

        # 5. Scaling (MinMax on Returns)
        scaler_y = Scaler() # Darts wrapper for MinMaxScaler
        target_train_scaled = scaler_y.fit_transform(target_train)
        target_test_scaled = scaler_y.transform(target_test)

        cov_train_scaled, cov_test_scaled = None, None
        if covariate_series:
            scaler_x = Scaler()
            cov_train_scaled = scaler_x.fit_transform(cov_train)
            cov_test_scaled = scaler_x.transform(cov_test)

        # 6. Train TFT
        model = TFTModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=output_chunk_length,
            hidden_size=TFT_CONFIG["hidden_size"],
            lstm_layers=1,
            num_attention_heads=TFT_CONFIG["num_attention_heads"],
            dropout=TFT_CONFIG["dropout"],
            batch_size=TFT_CONFIG["batch_size"],
            n_epochs=TFT_CONFIG["n_epochs"],
            add_relative_index=True,
            loss_fn=torch.nn.MSELoss(),
            likelihood=None,
            random_state=42,
            optimizer_kwargs={"lr": TFT_CONFIG["learning_rate"]},
            pl_trainer_kwargs={
                "accelerator": "cpu",
                "enable_progress_bar": False,
                "enable_model_summary": False,
                "logger": False
            }
        )

        model.fit(target_train_scaled, past_covariates=cov_train_scaled, verbose=False)

        # 7. Historical Forecasts (Block Updates)
        full_target_scaled = target_train_scaled.append(target_test_scaled)
        full_cov_scaled = None
        if covariate_series:
            full_cov_scaled = cov_train_scaled.append(cov_test_scaled)

        current_stride = output_chunk_length

        forecast_list = model.historical_forecasts(
            series=full_target_scaled,
            past_covariates=full_cov_scaled,
            start=train_cutoff,
            forecast_horizon=output_chunk_length,
            stride=current_stride,
            retrain=False,
            verbose=False,
            last_points_only=False
        )

        # 8. Reconstruction Loop
        if isinstance(forecast_list, TimeSeries):
            forecast_list = [forecast_list]

        all_pred_values = []
        all_pred_dates = []

        for chunk in forecast_list:
            # A. Inverse Scale (Get Log Returns)
            chunk_diff_log = scaler_y.inverse_transform(chunk)
            chunk_dates = chunk_diff_log.time_index
            chunk_log_returns = chunk_diff_log.values().flatten()

            # B. Find Anchor Price (Real Log-Price from yesterday)
            start_date = chunk_dates[0]
            # Get index in the LOG series (before differencing)
            idx = ts_log.get_index_at_point(start_date)
            anchor_log_price = ts_log[idx-1].values()[0][0]

            # C. Cumulative Sum to reconstruct Log Prices
            chunk_log_prices = anchor_log_price + np.cumsum(chunk_log_returns)
            
            # D. Exp to get Real Prices
            chunk_prices = np.exp(chunk_log_prices)

            all_pred_values.extend(chunk_prices)
            all_pred_dates.extend(chunk_dates)

        # 9. Create Prediction DataFrame
        pred_df_stitched = pd.DataFrame({
            "date": pd.to_datetime(all_pred_dates),
            "predicted": all_pred_values
        })
        
        # Merge with Full Data
        full_df = df[["date", target_col]].copy()
        full_df.columns = ["date", "actual"]
        
        final_df = pd.merge(full_df, pred_df_stitched, on="date", how="left")
        
        # Add train_flag (Based on original non-differenced cutoff)
        # Note: differencing shifted data by 1, but we map by date so it's fine.
        train_end_date = target_series[train_cutoff].time_index[0] # Approximate
        # Better: use the actual date from target_train end
        train_end_date = target_train.end_time()
        
        final_df["train_flag"] = 0
        final_df.loc[final_df["date"] <= train_end_date, "train_flag"] = 1

        # 10. Metrics
        eval_df = final_df.dropna(subset=["predicted"])
        
        if len(eval_df) == 0:
            raise ValueError("No predictions generated.")

        y_true = eval_df["actual"].values
        y_pred = eval_df["predicted"].values

        mse_val = np.mean((y_true - y_pred) ** 2)
        mae_val = np.mean(np.abs(y_true - y_pred))
        mape_val = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

        return {
            "mse": mse_val, "mae": mae_val, "mape": mape_val, 
            "success": True, "prediction_df": final_df
        }

    except Exception as e:
        traceback.print_exc()
        return {"mse": np.inf, "mae": np.inf, "mape": np.inf, "success": False, "prediction_df": None}

# ===================================================================
# MAIN EXECUTION LOOP
# ===================================================================

results = []
run_count = 0
total_runs = len(DATASETS) * len(SCENARIOS) * len(COVARIATE_SETS)

print("=" * 90)
print(f"🚀 STARTING TFT LOG-RETURN EXPERIMENTS")
print("=" * 90)

for target_name, dataset_path in DATASETS.items():
    if not os.path.exists(dataset_path):
        continue
        
    df = pd.read_csv(dataset_path)
    df["date"] = pd.to_datetime(df["date"])
    print(f"\n📂 Loaded {target_name} dataset")

    for scenario_name, scenario_params in SCENARIOS.items():
        input_len = scenario_params["window"]
        output_len = scenario_params["horizon"]

        for cov_name, covariates in COVARIATE_SETS.items():
            run_count += 1

            if run_count < START_FROM_RUN:
                continue

            print(f"[{run_count}/{total_runs}] ▶ Target={target_name} | Scenario={scenario_name} | Covs={cov_name}")

            cols = ["date", f"close_{target_name.lower()}"]
            if covariates:
                cols += covariates
            scenario_df = df[cols].dropna().copy()

            metrics = train_and_evaluate_tft(
                scenario_df,
                target_col=f"close_{target_name.lower()}",
                covariate_cols=covariates,
                input_chunk_length=input_len,
                output_chunk_length=output_len
            )

            # Save results
            if metrics["success"]:
                safe_target = target_name.replace(" ", "_")
                safe_scenario = scenario_name.replace(" ", "_")
                safe_cov = cov_name.replace(" ", "_")

                pred_filename = f"tft_pred_{safe_target}_{safe_scenario}_{safe_cov}.csv"
                pred_path = os.path.join(PREDICTION_DIR, pred_filename)

                metrics["prediction_df"].to_csv(pred_path, index=False)

                new_result = {
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "target": target_name,
                    "scenario": scenario_name,
                    "window": input_len,
                    "horizon": output_len,
                    "covariate_set": cov_name,
                    "mse": metrics["mse"],
                    "mae": metrics["mae"],
                    "mape": metrics["mape"],
                    "success": True
                }
                results.append(new_result)
                print(f"   ✅ MAPE={metrics['mape']:.2f}% | MAE={metrics['mae']:.2f}")
            else:
                print("   ❌ FAILED")
                results.append({
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "target": target_name,
                    "scenario": scenario_name,
                    "covariate_set": cov_name,
                    "success": False
                })

            pd.DataFrame(results).to_csv(RESULTS_SUMMARY_PATH, index=False)

print("\n" + "=" * 90)
print(f"✅ ALL EXPERIMENTS COMPLETE!")
print(f"Summary Results: {RESULTS_SUMMARY_PATH}")
print("=" * 90)

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


🚀 STARTING TFT LOG-RETURN EXPERIMENTS

📂 Loaded IHSG dataset
[1/99] ▶ Target=IHSG | Scenario=Short-Term Sprint | Covs=None


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   ✅ MAPE=0.54% | MAE=37.90
[2/99] ▶ Target=IHSG | Scenario=Short-Term Sprint | Covs=Domestic


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

   ✅ MAPE=2.63% | MAE=30.75
[98/99] ▶ Target=Kompas100 | Scenario=Long-Term Marathon | Covs=Brent


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   ✅ MAPE=2.64% | MAE=30.94
[99/99] ▶ Target=Kompas100 | Scenario=Long-Term Marathon | Covs=WTI


`Trainer.fit` stopped: `max_epochs=100` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


   ✅ MAPE=2.63% | MAE=30.75

✅ ALL EXPERIMENTS COMPLETE!
Summary Results: C:\wilson\Thesis\Results\tft_results_summary_logret.csv
